# Registry Parsing with *python-registry*










In [7]:
from Registry import Registry  # Better use dfwinreg (part of log2timeline)
reg = Registry.Registry("./Vibranium-NTUSER.DAT")  # Load NTUSER.dat of SANS Challenge 2015

## Ein erstes Beispiel
Auslesen des Root-Keys und der ersten Ebene darunter

Wichtige Methoden: 
    - node.subkeys()
    - node.value()

In [8]:
def retrieve_keys(key, stop, depth=0):
    '''
    Traverses the registry tree and prints all found keys to stdout
    '''
    if depth < stop or stop == -1:
        print( "|*|---" * depth + key.name())

        for subkey in key.subkeys():
            retrieve_keys(subkey, stop, depth + 1)

            
# Prints root and its direct children
retrieve_keys(reg.root(), 2)

CMI-CreateHive{6A1C4018-979D-4291-A7DC-7AED1C75B67C}
|*|---AppEvents
|*|---Console
|*|---Control Panel
|*|---Environment
|*|---EUDC
|*|---Identities
|*|---Keyboard Layout
|*|---Network
|*|---Printers
|*|---Software
|*|---System


## Ausgabe des gesamten Hives

In [9]:
# Prints whole hive-file structure
retrieve_keys(reg.root(), -1, 0)

CMI-CreateHive{6A1C4018-979D-4291-A7DC-7AED1C75B67C}
|*|---AppEvents
|*|---|*|---EventLabels
|*|---|*|---|*|---.Default
|*|---|*|---|*|---ActivatingDocument
|*|---|*|---|*|---AppGPFault
|*|---|*|---|*|---BlockedPopup
|*|---|*|---|*|---CCSelect
|*|---|*|---|*|---ChangeTheme
|*|---|*|---|*|---Close
|*|---|*|---|*|---CriticalBatteryAlarm
|*|---|*|---|*|---DeviceConnect
|*|---|*|---|*|---DeviceDisconnect
|*|---|*|---|*|---DeviceFail
|*|---|*|---|*|---DisNumbersSound
|*|---|*|---|*|---EmptyRecycleBin
|*|---|*|---|*|---FaxBeep
|*|---|*|---|*|---FaxError
|*|---|*|---|*|---FaxLineRings
|*|---|*|---|*|---FaxSent
|*|---|*|---|*|---FeedDiscovered
|*|---|*|---|*|---HubOffSound
|*|---|*|---|*|---HubOnSound
|*|---|*|---|*|---HubSleepSound
|*|---|*|---|*|---LowBatteryAlarm
|*|---|*|---|*|---MailBeep
|*|---|*|---|*|---Maximize
|*|---|*|---|*|---MenuCommand
|*|---|*|---|*|---MenuPopup
|*|---|*|---|*|---Minimize
|*|---|*|---|*|---MisrecoSound
|*|---|*|---|*|---MoveMenuItem
|*|---|*|---|*|---Navigating
|

|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 2
|*|---|*|---|*|---|*|---|*|---|*|---Mystify
|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 1
|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 2
|*|---|*|---|*|---|*|---|*|---|*|---Ribbons
|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 1
|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 2
|*|---|*|---|*|---|*|---|*|---|*|---ssText3d
|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 1
|*|---|*|---|*|---|*|---|*|---|*|---|*|---Screen 2
|*|---|*|---|*|---|*|---|*|---Shell Extensions
|*|---|*|---|*|---|*|---|*|---|*|---Cached
|*|---|*|---|*|---|*|---|*|---Sidebar
|*|---|*|---|*|---|*|---|*|---|*|---Settings
|*|---|*|---|*|---|*|---|*|---Telephony
|*|---|*|---|*|---|*|---|*|---|*|---HandoffPriorities
|*|---|*|---|*|---|*|---|*|---|*|---|*|---MediaModes
|*|---|*|---|*|---|*|---|*|---ThemeManager
|*|---|*|---|*|---|*|---|*|---Themes
|*|---|*|---|*|---|*|---|*|---|*|---DefaultVisualStyleOff
|*|---|*|---|*|---|*|---|*|---|*|---DefaultVisualSty

## Suchen von Schlüsseln

In [11]:
# level order tree traversal
def find_key(nodes, key):

    if len(nodes) == 0:
        return None
    new_nodes = []

    for n in nodes:
        if key in n.name():
            return n
        else:
            new_nodes += n.subkeys()

    return find_key(new_nodes, key)


search_key = r"Sec"
k = find_key([reg.root()], search_key)
print(k)



Registry Key CMI-CreateHive{6A1C4018-979D-4291-A7DC-7AED1C75B67C}\AppEvents\EventLabels\SecurityBand with 2 values and 0 subkeys


In [12]:
search_key = r"TypedURLs"
k = find_key([reg.root()], search_key)
print(k)

Registry Key CMI-CreateHive{6A1C4018-979D-4291-A7DC-7AED1C75B67C}\Software\Microsoft\Internet Explorer\TypedURLs with 2 values and 0 subkeys


## Extraktion von zugehörigen Werten

In [16]:
def get_leafs(node, leafs_to_populate):
    '''
    Retrieves all subkeys
    '''
    for n in node.subkeys():
        if len(n.subkeys()) == 0:
            leafs_to_populate.append(n)
        else:
            get_leafs(n, leafs_to_populate)
            
def extract_values(key):
    '''
    Retrieves all values under a given key
    '''
    leafs = []
    get_leafs(key, leafs)

    values = {}

    # Add values of top node
    if len(key.values()) > 0:
        vls = key.values()
        vls.sort(key=lambda x: x.name(), reverse=False)
        values[key] = vls

    # Add values of subnodes
    for l in leafs:
        if len(l.values()) > 0:
            entries = l.values()
            entries.sort(key=lambda x: x.name(), reverse=False)
            values[l] = entries
            

    return values

data_dict = extract_values(k)
print(data_dict)


{<Registry.Registry.RegistryKey object at 0x7fbfb80ef128>: [<Registry.Registry.RegistryValue object at 0x7fbfb8109f98>, <Registry.Registry.RegistryValue object at 0x7fbfb8109828>]}


## Verarbeitung der Werte

In [17]:
def convert_binary_data(data):
    data = data[::2]
    data = data[:data.find(b'\x00')]
    
    return data

In [18]:
result = ""
for k in data_dict.keys():
    print(20*"---")
    print(str(k))

    for v in data_dict[k]:
        # Prints datatype
        print("Datatype: " + v.value_type_str())

        # Prints("Name: ", v.name())
        if v.value_type() == Registry.RegNone:
            continue
        elif v.value_type() == Registry.RegBin:
            print(convert_binary_data(v.value()))
        else:
            print(v.value())

------------------------------------------------------------
Registry Key CMI-CreateHive{6A1C4018-979D-4291-A7DC-7AED1C75B67C}\Software\Microsoft\Internet Explorer\TypedURLs with 2 values and 0 subkeys
Datatype: RegSZ
http://199.73.28.114:53/
Datatype: RegSZ
http://go.microsoft.com/fwlink/?LinkId=69157


## Randnotiz: RegBin-Data
ASCII data represented by dwords

In [12]:
# Excerpt of Registry.py

def value_type(self):
    """
    Get the type of the value as an integer constant.

        One of:
         - RegSZ = 0x0001
         - RegExpandSZ = 0x0002
         - RegBin = 0x0003
         - RegDWord = 0x0004
         - RegMultiSZ = 0x0007
         - RegQWord = 0x000B
         - RegNone = 0x0000
         - RegBigEndian = 0x0005
...
    """
    return self._vkrecord.data_type()


b'D\x00o\x00w\x00n\x00l\x00o\x00a\x00d\x00s\x00\x00\x00d\x002\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Downloads.lnk\x00H\x00\x08\x00\x04\x00\xef\xbe\x00\x00\x00\x00\x00\x00\x00\x00*\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00D\x00o\x00w\x00n\x00l\x00o\x00a\x00d\x00s\x00.\x00l\x00n\x00k\x00\x00\x00\x1c\x00\x00\x00'


b'D\x00o\x00w\x00n\x00l\x00o\x00a\x00d\x00s\x00\x00\x00d\x002\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Downloads.lnk\x00H\x00\x08\x00\x04\x00\xef\xbe\x00\x00\x00\x00\x00\x00\x00\x00*\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00D\x00o\x00w\x00n\x00l\x00o\x00a\x00d\x00s\x00.\x00l\x00n\x00k\x00\x00\x00\x1c\x00\x00\x00'